In [1]:
%load_ext autoreload
%autoreload 2

import copy
import math
import random
import time
from collections import OrderedDict, defaultdict
from typing import Union, List
from utils import *

import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm
import torchvision.models as models

from torchprofile import profile_macs
# from torch.nn.parallel import DistributedDataParallel as DDP
# import torch.distributed as dist
from torch.nn.parallel import DataParallel


assert torch.cuda.is_available(), \
"CUDA support is not available."

/home/soheil/miniconda3/envs/hardwareAcc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
# set device 
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#set global device
# torch.cuda.set_device(device)
# dist.init_process_group(backend='nccl')

In [3]:
dataloader = get_dataloader("imagenet", batch_size=256)

In [4]:
base_model = models.vgg16(pretrained=True).cuda()

/home/soheil/miniconda3/envs/hardwareAcc/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/soheil/miniconda3/envs/hardwareAcc/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
evaluate_model(base_model, dataloader, count_nonzero_only=True)

eval:   0%|          | 0/20 [00:00<?, ?it/s]

model has test accuracy=68.06%
model has train accuracy=77.83%
model has size=527.79 MiB
model has macs=0.44 Gmacs
average inference time is 0.0012 seconds
model has 138.36 M parameters


In [10]:
compressed_model = copy.deepcopy(base_model)
compressed_model.classifier._modules[str(4)] = nn.LeakyReLU(negative_slope=nn.Parameter(torch.tensor(1), requires_grad=False), inplace=True)
compressed_model.classifier._modules[str(1)] = nn.LeakyReLU(negative_slope=nn.Parameter(torch.tensor(1), requires_grad=False), inplace=True)
collapse_layers_imagenet_vgg16(compressed_model, 4)
compressed_model.load_state_dict(torch.load('temp_best_model.pth'))
w0 = compressed_model.classifier._modules[str(0)].weight.data
w6 = compressed_model.classifier._modules[str(6)].weight.data
b0 = compressed_model.classifier._modules[str(0)].bias.data
b6 = compressed_model.classifier._modules[str(6)].bias.data
# model.classifier._modules[str(relu_id + 2)].weight.data = torch.matmul(w3, w0)
compressed_model.classifier._modules[str(6)].weight.data = torch.matmul(w6, w0)
# model.classifier._modules[str(relu_id + 2)].bias.data = torch.matmul(w3, b0) + b3
compressed_model.classifier._modules[str(6)].bias.data = torch.matmul(w6, b0) + b6

# model.classifier._modules[str(relu_id - 1)] = nn.Identity()
# model.classifier._modules[str(relu_id)] = nn.Identity()
# model.classifier._modules[str(relu_id + 1)] = nn.Identity()
compressed_model.classifier._modules[str(0)] = nn.Identity()
compressed_model.classifier._modules[str(1)] = nn.Identity()
compressed_model.classifier._modules[str(2)] = nn.Identity()

<All keys matched successfully>

In [11]:
print(compressed_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
evaluate_model(compressed_model, dataloader, count_nonzero_only=True)

model has test accuracy=62.72%
model has train accuracy=71.67%
model has size=151.84 MiB
model has macs=0.34 Gmacs
average inference time is 0.0007 seconds
model has 39.80 M parameters


In [15]:
print(base_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1